In [1]:
!pip install torch==1.12.1
!pip install torchtext==0.13.1
!pip install torchdata==0.4.1

  Using cached torch-1.12.0-cp39-none-macosx_10_9_x86_64.whl (133.6 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.12.0 which is incompatible.
torchdata 0.4.1 requires torch==1.12.1, but you have torch 1.12.0 which is incompatible.
  Using cached torch-1.12.1-cp39-none-macosx_10_9_x86_64.whl (133.8 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0
    Uninstalling torch-1.12.0:
      Successfully uninstalled torch-1.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.0 requir

In [2]:
import math
import time

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

torch.use_deterministic_algorithms(True)

In [3]:
class Transformer(nn.Module):
    def __init__(self, num_token, num_inputs, num_heads, num_hidden, num_layers, dropout=0.3):
        super(Transformer, self).__init__()
        self.model_name = 'transformer'
        self.position_enc = PosEnc(num_inputs, dropout)
        layers_enc = TransformerEncoderLayer(num_inputs, num_heads, num_hidden, dropout)
        self.enc_transformer = TransformerEncoder(layers_enc, num_layers)
        self.enc = nn.Embedding(num_token, num_inputs)
        self.num_inputs = num_inputs
        self.dec = nn.Linear(num_inputs, num_token)
        self.init_params()

    def init_params(self):
        initial_rng = 0.12
        self.enc.weight.data.uniform_(-initial_rng, initial_rng)
        self.dec.bias.data.zero_()
        self.dec.weight.data.uniform_(-initial_rng, initial_rng)

    def forward(self, source, mask_source):
        source = self.enc(source) * math.sqrt(self.num_inputs)
        source = self.position_enc(source)
        op = self.enc_transformer(source, mask_source)
        op = self.dec(op)
        return op

def gen_sqr_nxt_mask(size):
    msk = torch.triu(torch.ones(size, size) * float('-inf'), diagonal=1)
    return msk

In [4]:
class PosEnc(nn.Module):
    def __init__(self, d_m, dropout=0.2, size_limit=5000):
        super(PosEnc, self).__init__()
        self.dropout = nn.Dropout(dropout)
        p_enc = torch.zeros(size_limit, 1, d_m)
        pos = torch.arange(size_limit, dtype=torch.float).unsqueeze(1)
        divider = torch.exp(torch.arange(0, d_m, 2).float() * (-math.log(10000.0) / d_m))
        p_enc[:, 0, 0::2] = torch.sin(pos * divider)
        p_enc[:, 0, 1::2] = torch.cos(pos * divider)
        self.register_buffer('p_enc', p_enc)
        
    def forward(self, x):
        return self.dropout(x + self.p_enc[:x.size(0)])

In [5]:
tr_iter = WikiText2(split='train')
tkzer = get_tokenizer('basic_english')
vocabulary = build_vocab_from_iterator(map(tkzer, tr_iter), specials=['<unk>'])
vocabulary.set_default_index(vocabulary['<unk>'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def process_data(raw_text):
    numericalised_text = [torch.tensor(vocabulary(tkzer(text)), dtype=torch.long) for text in raw_text]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, numericalised_text)))

tr_iter, val_iter, te_iter = WikiText2()
training_text = process_data(tr_iter)
validation_text = process_data(val_iter)
testing_text = process_data(te_iter)

def gen_batches(text_dataset, batch_size):
    num_batches = text_dataset.size(0) // batch_size
    text_dataset = text_dataset[:num_batches * batch_size]
    text_dataset = text_dataset.view(batch_size, num_batches).t().contiguous()
    return text_dataset.to(device)

training_batch_size = 32
evaluation_batch_size = 16

training_data = gen_batches(training_text, training_batch_size)
validation_data = gen_batches(validation_text, evaluation_batch_size)
testing_data = gen_batches(testing_text, evaluation_batch_size)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4.48M/4.48M [00:00<00:00, 5.08MB/s]


In [6]:
max_seq_len = 64
def return_batch(src, k):
    sequence_length = min(max_seq_len, len(src) - 1 - k)
    sequence_data = src[k:k+sequence_length]
    sequence_label = src[k+1:k+1+sequence_length].reshape(-1)
    return sequence_data, sequence_label

In [7]:
num_tokens = len(vocabulary) # vocabulary size
embedding_size = 256 # dimension of embedding layer
num_hidden_params = 256 # transformer encoder's hidden (feed forward) layer dimension
num_layers = 2 # num of transformer encoder layers within transformer encoder
num_heads = 2 # num of heads in (multi head) attention models
dropout = 0.25 # value (fraction) of dropout
loss_func = nn.CrossEntropyLoss()
lrate = 4.0 # learning rate
transformer_model = Transformer(num_tokens, embedding_size, num_heads, num_hidden_params, num_layers, 
                                     dropout).to(device)
optim_module = torch.optim.SGD(transformer_model.parameters(), lr=lrate)
sched_module = torch.optim.lr_scheduler.StepLR(optim_module, 1.0, gamma=0.88)

In [8]:
def train_model():
    transformer_model.train()
    loss_total = 0.
    time_start = time.time()
    mask_source = gen_sqr_nxt_mask(max_seq_len).to(device)
    num_batches = len(training_data) // max_seq_len
    for b, i in enumerate(range(0, training_data.size(0) - 1, max_seq_len)):
        train_data_batch, train_label_batch = return_batch(training_data, i)
        sequence_length = train_data_batch.size(0)
        if sequence_length != max_seq_len:  # only on last batch
            mask_source = mask_source[:sequence_length, :sequence_length]
        op = transformer_model(train_data_batch, mask_source)
        loss_curr = loss_func(op.view(-1, num_tokens), train_label_batch)
        optim_module.zero_grad()
        loss_curr.backward()
        torch.nn.utils.clip_grad_norm_(transformer_model.parameters(), 0.6)
        optim_module.step()

        loss_total += loss_curr.item()
        interval = 100
        if b % interval == 0 and b > 0:
            loss_interval = loss_total / interval
            time_delta = time.time() - time_start
            print(f"epoch {ep}, {b}/{len(training_data)//max_seq_len} batches, training loss {loss_interval:.2f}, training perplexity {math.exp(loss_interval):.2f}")
            loss_total = 0
            time_start = time.time()

def eval_model(eval_model_obj, eval_data_source):
    eval_model_obj.eval() 
    loss_total = 0.
    mask_source = gen_sqr_nxt_mask(max_seq_len).to(device)
    with torch.no_grad():
        for j in range(0, eval_data_source.size(0) - 1, max_seq_len):
            eval_data, eval_label = return_batch(eval_data_source, j)
            sequence_length = eval_data.size(0)
            if sequence_length != max_seq_len:
                mask_source = mask_source[:sequence_length, :sequence_length]
            op = eval_model_obj(eval_data, mask_source)
            op_flat = op.view(-1, num_tokens)
            loss_total += sequence_length * loss_func(op_flat, eval_label).item()
    return loss_total / (len(eval_data_source) - 1)

In [9]:
min_validation_loss = float("inf")
eps = 25
best_model_so_far = None

for ep in range(1, eps + 1):
    ep_time_start = time.time()
    train_model()
    validation_loss = eval_model(transformer_model, validation_data)
    print()
    print(f"epoch {ep:}, validation loss {validation_loss:.2f}, validation perplexity {math.exp(validation_loss):.2f}")
    print()

    if validation_loss < min_validation_loss:
        min_validation_loss = validation_loss
        best_model_so_far = transformer_model

    sched_module.step()

epoch 1, 100/1000 batches, training loss 8.81, training perplexity 6724.85
epoch 1, 200/1000 batches, training loss 7.35, training perplexity 1555.26
epoch 1, 300/1000 batches, training loss 6.90, training perplexity 991.85
epoch 1, 400/1000 batches, training loss 6.67, training perplexity 792.05
epoch 1, 500/1000 batches, training loss 6.54, training perplexity 694.65
epoch 1, 600/1000 batches, training loss 6.39, training perplexity 597.00
epoch 1, 700/1000 batches, training loss 6.34, training perplexity 569.06
epoch 1, 800/1000 batches, training loss 6.21, training perplexity 498.15
epoch 1, 900/1000 batches, training loss 6.18, training perplexity 485.01
epoch 1, 1000/1000 batches, training loss 6.16, training perplexity 472.72

epoch 1, validation loss 5.87, validation perplexity 353.51

epoch 2, 100/1000 batches, training loss 6.07, training perplexity 430.66
epoch 2, 200/1000 batches, training loss 5.99, training perplexity 399.61
epoch 2, 300/1000 batches, training loss 5.91, 

epoch 11, 300/1000 batches, training loss 4.77, training perplexity 117.61
epoch 11, 400/1000 batches, training loss 4.76, training perplexity 117.09
epoch 11, 500/1000 batches, training loss 4.77, training perplexity 117.95
epoch 11, 600/1000 batches, training loss 4.76, training perplexity 116.57
epoch 11, 700/1000 batches, training loss 4.76, training perplexity 117.14
epoch 11, 800/1000 batches, training loss 4.63, training perplexity 102.52
epoch 11, 900/1000 batches, training loss 4.69, training perplexity 109.21
epoch 11, 1000/1000 batches, training loss 4.76, training perplexity 117.24

epoch 11, validation loss 5.14, validation perplexity 170.42

epoch 12, 100/1000 batches, training loss 4.78, training perplexity 119.25
epoch 12, 200/1000 batches, training loss 4.71, training perplexity 111.20
epoch 12, 300/1000 batches, training loss 4.72, training perplexity 111.81
epoch 12, 400/1000 batches, training loss 4.71, training perplexity 111.42
epoch 12, 500/1000 batches, training

epoch 21, 500/1000 batches, training loss 4.49, training perplexity 89.24
epoch 21, 600/1000 batches, training loss 4.49, training perplexity 88.83
epoch 21, 700/1000 batches, training loss 4.49, training perplexity 89.17
epoch 21, 800/1000 batches, training loss 4.37, training perplexity 78.77
epoch 21, 900/1000 batches, training loss 4.43, training perplexity 83.60
epoch 21, 1000/1000 batches, training loss 4.49, training perplexity 88.95

epoch 21, validation loss 5.08, validation perplexity 160.65

epoch 22, 100/1000 batches, training loss 4.53, training perplexity 92.97
epoch 22, 200/1000 batches, training loss 4.46, training perplexity 86.83
epoch 22, 300/1000 batches, training loss 4.48, training perplexity 88.53
epoch 22, 400/1000 batches, training loss 4.48, training perplexity 88.05
epoch 22, 500/1000 batches, training loss 4.48, training perplexity 88.57
epoch 22, 600/1000 batches, training loss 4.47, training perplexity 87.68
epoch 22, 700/1000 batches, training loss 4.48, 

In [10]:
testing_loss = eval_model(best_model_so_far, testing_data)
print(f"testing loss {testing_loss:.2f}, testing perplexity {math.exp(testing_loss):.2f}")

testing loss 5.00, testing perplexity 148.86


In [11]:
mdl_pth = './transformer.pth'
torch.save(best_model_so_far.state_dict(), mdl_pth)

In [12]:
# load the best trained model
transformer_cached = Transformer(num_tokens, embedding_size, num_heads, num_hidden_params, num_layers, 
                                     dropout).to(device)
transformer_cached.load_state_dict(torch.load(mdl_pth))

<All keys matched successfully>

In [78]:
ln = 5
sntc = 'They are _'
sntc_split = sntc.split()
torch.manual_seed(34)
mask_source = gen_sqr_nxt_mask(max_seq_len).to(device)
with torch.no_grad():
    for i in range(ln):
        sntc = ' '.join(sntc_split)
        txt_ds = Tensor(vocabulary(sntc_split)).unsqueeze(0).to(torch.long)
        num_b = txt_ds.size(0)
        txt_ds = txt_ds.narrow(0, 0, num_b)
        txt_ds = txt_ds.view(1, -1).t().contiguous().to(device)
        ev_X, _ = return_batch(txt_ds, i+1)
        sequence_length = ev_X.size(0)
        if sequence_length != max_seq_len:
            mask_source = mask_source[:sequence_length, :sequence_length]
        op = transformer_cached(ev_X, mask_source)
        op_flat = op.view(-1, num_tokens)
        res = vocabulary.get_itos()[op_flat.argmax(1)[0]]
        sntc_split.insert(-1, res)
print(sntc[:-2])

They are often used for the
